In [1]:
import numpy as np
import pandas as pd
from nilearn import datasets, image, input_data 

### Get Power 2011 coordinates from Nilearn and write them to a CSV.

In [2]:
power_2011 = datasets.fetch_coords_power_2011()

In [3]:
power_2011_df = pd.DataFrame.from_records(power_2011.rois)

In [4]:
power_2011_df.rename(index=str, columns={'roi':'roi_num'}, inplace=True)

In [6]:
!mkdir mni_2_t1_testing

In [7]:
# Write a csv for use by AFNI's 3dUndump
power_2011_df.to_csv('/home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_MNI_afni.csv', sep=',',
                    columns=['x', 'y', 'z', 'roi_num'], index=False)

In [8]:
power_2011_df['t'] = 0

In [9]:
# Write a csv for use by antsApplyTransformsToPoints
power_2011_df.to_csv('/home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_MNI_ants.csv', sep=',',
                    columns=['x', 'y', 'z', 't'], index=False)

### Generate an image of the ROIs (4mm radius spheres) in MNI space.

In [10]:
%%bash
3dUndump \
-master /home/despo/dlurie/Software/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz \
-prefix mni_2_t1_testing/power2011_MNI.nii.gz \
-srad 4 \
-xyz /home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_MNI_afni.csv \

++ 3dUndump: AFNI version=AFNI_16.2.16 (Sep  4 2016) [64-bit]
++ Starting to fill via -xyz coordinates
++ Total number of voxels filled = 67848
++ Wrote out dataset ./mni_2_t1_testing/power2011_MNI.nii.gz


In `power2011_MNI.nii.gz`, ROI #1 is located at the right Occipital Pole: MNI coordinate xyz=[25, -98, -12]

This does not match the MNI coordinate in the text file: xyz=[-25, -98, -12]

In [11]:
%%bash
3dUndump \
-master /home/despo/dlurie/Software/fsl/data/standard/MNI152_T1_1mm_brain.nii.gz \
-prefix mni_2_t1_testing/power2011_MNI_LPI.nii.gz \
-orient LPI \
-srad 4 \
-xyz /home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_MNI_afni.csv \

++ 3dUndump: AFNI version=AFNI_16.2.16 (Sep  4 2016) [64-bit]
++ Starting to fill via -xyz coordinates
++ Total number of voxels filled = 67848
++ Wrote out dataset ./mni_2_t1_testing/power2011_MNI_LPI.nii.gz


Specifying LPI coordinate ordering for `3dUndump` produces ROIs at the proper locations and MNI coordinates. 

ROI #1 is located at the left Occipital Pole, MNI coordinate xyz=[-25, -98, -12]

### Transform ROI image from MNI to T1 space.

In [12]:
%%bash
antsApplyTransforms \
-d 3 \
-r /home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_out/derivatives/sub-185/anat/sub-185_T1w_preproc.nii.gz \
--input mni_2_t1_testing/power2011_MNI_LPI.nii.gz \
-o mni_2_t1_testing/power2011_sub185_MNI_2_T1.nii.gz \
-n NearestNeighbor \
-t [/home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_work/workflow_enumerator/185/t1w_preprocessing/T1_2_MNI_Registration/ants_t1_to_mni0GenericAffine.mat,1] \
-t [/home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_work/workflow_enumerator/185/t1w_preprocessing/T1_2_MNI_Registration/ants_t1_to_mni1InverseWarp.nii.gz,0]

### Get mapping between MNI coordinates and MNI-ITK coordinates.

In [14]:
%%bash
ImageMath 3 mni_2_t1_testing/testvox_MNI.txt LabelStats mni_2_t1_testing/testvox_MNI.nii.gz

`testvox_MNI.nii.gz` contains a single voxel at MNI coordiante xyz=[-26, -99, -11]

In [15]:
!cat mni_2_t1_testing/testvox_MNI.txt

x,y,z,t,label,mass,volume,count
26,99,-11,0,1,0,1,1


To convert MNI coordinates to MNI-ITK coordinates, we must multiply x and y by -1.

### Transform Power 2011 coordinates from MNI to MNI/ITK space and write to a CSV.

In [16]:
power_2011_MNI_ITK_df = power_2011_df.copy()

In [17]:
power_2011_MNI_ITK_df.head()

,roi_num,x,y,z,t
0,1,-25,-98,-12,0
1,2,27,-97,-13,0
2,3,24,32,-18,0
3,4,-56,-45,-24,0
4,5,8,41,-24,0


In [18]:
power_2011_MNI_ITK_df['x'] = power_2011_MNI_ITK_df['x'] * -1
power_2011_MNI_ITK_df['y'] = power_2011_MNI_ITK_df['y'] * -1

In [19]:
power_2011_MNI_ITK_df.head()

,roi_num,x,y,z,t
0,1,25,98,-12,0
1,2,-27,97,-13,0
2,3,-24,-32,-18,0
3,4,56,45,-24,0
4,5,-8,-41,-24,0


In [20]:
power_2011_MNI_ITK_df['t'] = 0

In [21]:
# Write a csv for use by antsApplyTransformsToPoints
power_2011_MNI_ITK_df.to_csv('/home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_MNI_ITK_ants.csv', sep=',',
                    columns=['x', 'y', 'z', 't'], index=False)

### Transform Power 2011 coordinates from MNI-ITK to T1-ITK space for patient 185

In [22]:
%%bash
antsApplyTransformsToPoints \
-d 3 \
-i /home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_MNI_ITK_ants.csv \
-o /home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_T1_ITK.csv \
-t [/home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_out/derivatives/sub-185/anat/sub-185_T1w_target-MNI152NLin2009cAsym_warp.nii.gz, 0] \
-t [/home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_out/derivatives/sub-185/anat/sub-185_T1w_target-MNI152NLin2009cAsym_affine.mat, 0]

### Get mapping between T1-ITK coordinates and T1 coordinates.

In `power2011_sub185_MNI_2_T1.nii.gz`, ROI #1 is located at the left Occipital Pole, xyz~[-22.32, -43.84, -53.84].

We'll place a test voxel at this location to get the mapping.

In [23]:
%%bash
ImageMath 3 mni_2_t1_testing/testvox_T1.txt LabelStats mni_2_t1_testing/testvox_T1.nii.gz

In [24]:
!cat mni_2_t1_testing/testvox_T1.txt

x,y,z,t,label,mass,volume,count
22.3184,43.8437,-53.8403,0,1,0,1,1


The output of LabelStats gives the same coordinate for ROI #1 as listed in the coordinates transformed by `antsApplyTransformsToPoints`.

To convert T1-ITK coordinates to T1 coordinates, we must multiply x and y by -1.

### Transform Power 2011 coordinates from T1-ITK to T1 space and write to a CSV.

In [25]:
power_2011_T1_ITK_df = pd.read_csv('/home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_T1_ITK.csv')

In [26]:
power_2011_T1_ITK_df.head()

,x,y,z,t
0,21.9834,43.3662,-53.7220,0
1,-25.5121,49.1142,-54.7395,0
2,-38.6173,-71.2065,-31.3740,0
3,41.5211,-15.5412,-53.9657,0
4,-25.8182,-83.1194,-33.1153,0


In [27]:
power_2011_T1_df = power_2011_T1_ITK_df.copy()

In [28]:
power_2011_T1_df['x'] = power_2011_T1_df['x'] * -1
power_2011_T1_df['y'] = power_2011_T1_df['y'] * -1

In [29]:
power_2011_T1_df.head()

,x,y,z,t
0,-21.9834,-43.3662,-53.7220,0
1,25.5121,-49.1142,-54.7395,0
2,38.6173,71.2065,-31.3740,0
3,-41.5211,15.5412,-53.9657,0
4,25.8182,83.1194,-33.1153,0


In [30]:
power_2011_T1_df['roi_num'] = power_2011_T1_df.index + 1

In [31]:
# Write a csv for use by AFNI's 3dUndump
power_2011_T1_df.to_csv('/home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_T1_afni.csv', sep=',',
                    columns=['x', 'y', 'z', 'roi_num'], index=False)

### Generate an image of the ROIs (4mm radius spheres) in T1 space.

In [32]:
%%bash
3dUndump \
-master /home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_out/derivatives/sub-185/anat/sub-185_T1w_preproc.nii.gz \
-prefix mni_2_t1_testing/power2011_T1.nii.gz \
-srad 4 \
-xyz /home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_T1_afni.csv \

++ 3dUndump: AFNI version=AFNI_16.2.16 (Sep  4 2016) [64-bit]
*+ WARNING:   If you are performing spatial transformations on an oblique dset, 
  such as /home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_out/derivatives/sub-185/anat/sub-185_T1w_preproc.nii.gz,
  or viewing/combining it with volumes of differing obliquity,
  you should consider running: 
     3dWarp -deoblique 
  on this and  other oblique datasets in the same session.
 See 3dWarp -help for details.
++ Oblique dataset:/home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_out/derivatives/sub-185/anat/sub-185_T1w_preproc.nii.gz is 16.900002 degrees from plumb.
++ Starting to fill via -xyz coordinates
*+ WARNING: File /home/despo/dlurie/Projects/despolab_lesion/mni_2_t1_testing/power_coords_T1_afni.csv line 157: y coord=49.9985 is outside -190.259 .. 49.7429
++ Total number of voxels filled = 67253
++ Wrote out dataset ./mni_2_t1_testing/power2011_T1.nii.gz


In `power2011_T1.nii.gz`, ROI #1 is located outside the brain, behind the left Occipital Pole, coordinate xyz=[-22.32, -87.96, -59.53]

This does not match the MNI coordinate in the text file or the warped image: xyz=[-21.98,-43.36,-53.72]

Because ROI #1 is on the correct size of the brain, and the spatial relationship of the ROIs seems to make sense other than their offset from the brain, I am guessing the problem has to do with how AFNI handles oblique datasets.

### Try creating spheres in Nilearn

In [33]:
import nibabel as nib
from nilearn.input_data import NiftiMasker
from nilearn.input_data.nifti_spheres_masker import  \
    _apply_mask_and_get_affinity

In [34]:
brain_mask = '/home/despo/dlurie/Projects/despolab_lesion/pipeline_testing/test_out/derivatives/sub-185/anat/sub-185_T1w_brainmask.nii.gz'
brain_mask_img = nib.load(brain_mask)

In [35]:
seeds = power_2011_T1_df.loc[:,'x':'z'].values
radius = 4

In [36]:
# apply sphere masking on the mask
_, X = _apply_mask_and_get_affinity(seeds, brain_mask_img, radius=radius,
                                    allow_overlap=True,
                                    mask_img=brain_mask_img)

In [37]:
# inverse-transform X that contains masked images with spheres
masker = NiftiMasker(mask_img=brain_mask)
masker.fit()
img = masker.inverse_transform(np.sum(X, axis=0).A.ravel())

In [38]:
img.to_filename('mni_2_t1_testing/nilearn_spheres_test.nii.gz')